In [28]:
import sys
import lzma
from s2clientprotocol.sc2api_pb2 import Response, ResponseObservation
from MapAnalyzer.utils import import_bot_instance
from MapAnalyzer import MapData
import pickle
import numpy as np
import matplotlib.pyplot as plt
sys.path.append("src")
sys.path.append("src/ares")
from rust_helpers import number_of_chokes_pathed_through, terrain_flood_fill
%load_ext line_profiler
%load_ext Cython

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler
The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [2]:
BERLINGRAD = "tests/pickle_data/BerlingradAIE.xz"

In [3]:
with lzma.open(BERLINGRAD, "rb") as f:
    raw_game_data, raw_game_info, raw_observation = pickle.load(f)
bot = import_bot_instance(raw_game_data, raw_game_info, raw_observation)
data = MapData(bot)

2023-05-06 14:28:56.771 | INFO     | MapAnalyzer.MapData:__init__:122 - dev Compiling Berlingrad AIE 
 Version dev Map Compilation Progress : 0.2it [00:00,  1.07it/s]


In [4]:
%%cython
from cython cimport boundscheck, wraparound

cdef unsigned int euclidean_distance_squared_int((unsigned int, unsigned int) p1, (unsigned int, unsigned int) p2):
    return (p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2

cpdef set flood_fill(
    (unsigned int, unsigned int) start_point, 
    const unsigned char[:, :] terrain_grid, 
    const unsigned char[:, :] pathing_grid, 
    unsigned int max_distance, 
    set choke_points
):
    cdef:
        unsigned int terrain_height = terrain_grid[start_point[0], start_point[1]]
        unsigned int pathing_value = pathing_grid[start_point[0], start_point[1]]
        set filled_points = set()

    # Only continue if we can get a height for the starting point
    if not terrain_height:
        return filled_points
    
    if pathing_value != 1:
        return filled_points
        
    grid_flood_fill(start_point, terrain_grid, pathing_grid, terrain_height, filled_points, start_point, max_distance, choke_points)
    return filled_points

cdef set grid_flood_fill(
    (unsigned int, unsigned int) point, 
    const unsigned char[:, :] terrain_grid, 
    const unsigned char[:, :] pathing_grid, 
    unsigned int target_val, 
    set current_vec, 
    (unsigned int, unsigned int) start_point, 
    unsigned int max_distance, 
    set choke_points):
    cdef:
        unsigned int terrain_height = terrain_grid[start_point[0], start_point[1]]
        unsigned int pathing_value = pathing_grid[start_point[0], start_point[1]]
    # Check that we haven't already added this point.
    if point in current_vec:
        return current_vec

    # Check that this point isn't too far away from the start
    if euclidean_distance_squared_int(point, start_point) > max_distance ** 2:
        return current_vec

    if point in choke_points:
        return current_vec

    terrain_height = terrain_grid[point[0], point[1]]
    pathing_value = pathing_grid[point[0], point[1]]
    if terrain_height != target_val or pathing_value != 1:
        return current_vec
    
    current_vec.add(point)
    grid_flood_fill((point[0]+1, point[1]), terrain_grid, pathing_grid, terrain_height, current_vec, start_point, max_distance, choke_points)
    grid_flood_fill((point[0]-1, point[1]), terrain_grid, pathing_grid, terrain_height, current_vec, start_point, max_distance, choke_points)
    grid_flood_fill((point[0], point[1]+1), terrain_grid, pathing_grid, terrain_height, current_vec, start_point, max_distance, choke_points)
    grid_flood_fill((point[0], point[1]-1), terrain_grid, pathing_grid, terrain_height, current_vec, start_point, max_distance, choke_points)

In [5]:
%%cython
from cython cimport boundscheck, wraparound
# The boundscheck(False) decorator tells Cython that we know that we will not access elements outside the bounds of the units list, which allows for faster indexing. 
# The wraparound(False) decorator tells Cython that we know that we will not use negative indices to access elements of the list, which also allows for faster indexing.
@boundscheck(False)
@wraparound(False)
cpdef ((float, float), (float, float)) get_bounding_box(set coordinates):
    cdef:
        float x_min = 9999.0
        float x_max = 0.0
        float x_val = 0.0
        float y_min = 9999.0
        float y_max = 0.0
        float y_val = 0.0
        int start = 0
        int stop = len(coordinates)
        (float, float) position
    for i in range(start, stop):
        position = coordinates.pop()
        x_val = position[0]
        y_val = position[1]
        if x_val < x_min:
            x_min = x_val
        if x_val > x_max:
            x_max = x_val
        if y_val < y_min:
            y_min = y_val
        if y_val > y_max:
            y_max = y_val
    return (x_min, x_max), (y_min, y_max)

In [29]:
%%cython

from scipy.signal import convolve2d
import numpy as np
cimport numpy as np

cpdef find_building_locations(
    np.ndarray[np.uint8_t, ndim=2] kernel,
    int x_stride,
    int y_stride,
    (int, int) x_bounds,
    (int, int) y_bounds,
    const unsigned char[:, :] creep_grid,
    const unsigned char[:, :] placement_grid,
    const unsigned char[:, :] vision_grid,
    const float[:, :] pathing_grid,
    const unsigned char[:, :] points_to_avoid_grid,
    unsigned int building_width,
    unsigned int building_height,
    bint include_addon = False,
):
    """
    Use a convolution pass to find all possible building locations in an area
    See full docs in `placement_solver.pyi`
    
    Note - removed the padding argument since np.ones(dims) works
    """
    cdef:
        unsigned int i = 0
        unsigned int j = 0
        unsigned int _x = 0
        unsigned int _y = 0
        unsigned int valid_idx = 0
        float x, y
        float weighted_x, weighted_y
        int x_min = x_bounds[0]
        int x_max = x_bounds[1]
        int y_min = y_bounds[0]
        int y_max = y_bounds[1]
        unsigned char[:, :] to_convolve = np.ones((x_max - x_min + 1, y_max - y_min + 1), dtype=np.uint8)
        (float, float) [500] valid_spots
        (float, float) center
        float half_width = building_width / 2

    # use an inverted placement grid and check that all tiles are placeable
    for i in range(x_min, x_max + 1):
        for j in range(y_min, y_max + 1):
            if points_to_avoid_grid[j][i] == 0 and creep_grid[j][i] == 0 and placement_grid[j][i] == 1 and pathing_grid[j][i] == 1:
                to_convolve[i - x_min][j - y_min] = 0

    cdef unsigned char[:, :] result = convolve2d(to_convolve, kernel, mode="valid")

    for i in range(0, result.shape[0], x_stride):
        for j in range(0, result.shape[1], y_stride):
            # check that the result is 0 now- if any tile is unplaceable, the total will be >= 1
            if result[i][j] == 0:
                x = i + x_min + half_width
                y = j + y_min + half_width

                if include_addon:
                    _x = int(x + 3.5)
                    _y = int(y - 1.5)
                    # check placement of edge of addon
                    if placement_grid[_y][_x] == 0 or creep_grid[_y][_x] == 1 or pathing_grid[_y][_x] == 0 or points_to_avoid_grid[_y][_x] == 1:
                        continue
                    if placement_grid[_y-1][_x] == 0 or creep_grid[_y-1][_x] == 1 or pathing_grid[_y-1][_x] == 0 or points_to_avoid_grid[_y-1][_x] == 1:
                        continue
                    if placement_grid[_y+1][_x] == 0 or creep_grid[_y+1][_x] == 1 or pathing_grid[_y+1][_x] == 0 or points_to_avoid_grid[_y+1][_x] == 1:
                        continue
                # valid building placement is building center, so add half to x and y
                valid_spots[valid_idx][0] = x
                valid_spots[valid_idx][1] = y
                valid_idx += 1

            if valid_idx >= 500:
                # we've reached the maximum number of valid spots
                break

    if valid_idx == 0:
        return [], (0, 0)

    i = 0
    for i in range(0, valid_idx):
        weighted_x += valid_spots[i][0]
        weighted_y += valid_spots[i][1]

    center = (weighted_x / valid_idx, weighted_y / valid_idx)

    return list(valid_spots)[:valid_idx], center

In [7]:
start_point=bot.enemy_start_locations[0].rounded
terrain_grid=bot.game_info.terrain_height.data_numpy.T
pathing_grid=data.get_pyastar_grid().astype(np.uint8)
max_distance=20
choke_points=set(
        [point for ch in data.map_chokes for point in ch.points]
    )

In [8]:
raw_x_bounds, raw_y_bounds = get_bounding_box(flood_fill(start_point, terrain_grid, pathing_grid, max_distance, choke_points))

In [34]:
"""
3x3 kernel, use standard strides
"""
kernel: np.ndarray = np.ones(
            (3, 3), dtype=np.uint8
        )

points_to_avoid_grid = np.zeros(bot.game_info.placement_grid.data_numpy.shape, dtype=np.uint8)

# avoid within 5.5 distance of base location
start_x = int(bot.townhalls[0].position.x - 5.5)
start_y = int(bot.townhalls[0].position.y - 5.5)

points_to_avoid_grid[start_x:start_x+11, start_y:start_y+11] = 1

production_positions, _center = find_building_locations(
    kernel=kernel, 
    x_stride=5,
    y_stride=3,
    x_bounds=raw_x_bounds, 
    y_bounds=raw_y_bounds, 
    creep_grid=bot.state.creep.data_numpy, 
    placement_grid=bot.game_info.placement_grid.data_numpy, 
    vision_grid=bot.state.visibility.data_numpy, 
    pathing_grid=data.get_pyastar_grid(),
    # bot.game_info.pathing_grid.data_numpy, 
    points_to_avoid_grid=points_to_avoid_grid, 
    building_width=3, 
    building_height=3,
    include_addon=False,
)
print("Production Positions:")
production_positions

Production Positions:


[(24.5, 130.5),
 (29.5, 130.5),
 (39.5, 121.5),
 (39.5, 124.5),
 (39.5, 127.5),
 (39.5, 130.5),
 (44.5, 118.5),
 (44.5, 121.5),
 (44.5, 124.5),
 (44.5, 133.5),
 (49.5, 121.5),
 (49.5, 124.5),
 (49.5, 127.5),
 (49.5, 133.5)]

In [33]:
"""
8x8 kernel - much stricter on placements so reduce stride
"""
kernel: np.ndarray = np.ones(
            (8, 8), dtype=np.uint8
        )

points_to_avoid_grid = np.zeros(bot.game_info.placement_grid.data_numpy.shape, dtype=np.uint8)

# avoid within 5.5 distance of base location
start_x = int(bot.townhalls[0].position.x - 5.5)
start_y = int(bot.townhalls[0].position.y - 5.5)

points_to_avoid_grid[start_x:start_x+11, start_y:start_y+11] = 1

production_positions, _center = find_building_locations(
    kernel=kernel, 
    x_stride=1,
    y_stride=1,
    x_bounds=raw_x_bounds, 
    y_bounds=raw_y_bounds, 
    creep_grid=bot.state.creep.data_numpy, 
    placement_grid=bot.game_info.placement_grid.data_numpy, 
    vision_grid=bot.state.visibility.data_numpy, 
    pathing_grid=data.get_pyastar_grid(),
    # bot.game_info.pathing_grid.data_numpy, 
    points_to_avoid_grid=points_to_avoid_grid, 
    building_width=3, 
    building_height=3,
    include_addon=False,
)
print("Production Positions:")
production_positions

Production Positions:


[(44.5, 119.5), (44.5, 120.5), (44.5, 121.5)]